In [181]:
import pandas as pd 
from nltk.corpus import stopwords
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import numpy as np
from numpy import argmax
from torch.nn.functional import softmax

In [120]:
model_name = "ProsusAI/finbert"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
df = pd.read_csv('stockNews.csv')
for col in df.columns:
    col = col.strip
df.drop('Unnamed: 0', axis=1, inplace=True)
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)
df

In [185]:
def get_sentiment(news):
    inputs = tokenizer(news, return_tensors="pt", padding=True, truncation=True, max_length=512)
    
    with torch.no_grad():  
        predictions = model(**inputs)
    
    prob = softmax(predictions.logits, dim=-1)
    sentiment_list = ['positive', 'neutral', 'negative']
    sentiment_score = max(prob[0].tolist())
    sentiment = sentiment_list[prob[0].tolist().index(sentiment_score)]
    
    return sentiment, sentiment_score


In [191]:
def sliding_window(text, tokenizer, max_length=512, overlap=50):

    tokenized_text = tokenizer.encode_plus(text, return_tensors='pt', add_special_tokens=True)
    input_ids = tokenized_text['input_ids'].squeeze().numpy().tolist()
    
    total_length = len(input_ids)
    step_size = max_length - overlap
    chunks = []
    
    for start in range(0, total_length, step_size):
        end = start + max_length
        chunk = input_ids[start:end]
        
        chunk = [tokenizer.cls_token_id] + chunk + [tokenizer.sep_token_id]
        chunks.append(chunk)
    
    return chunks


In [195]:
def process_chunks(chunks, tokenizer, model):
    model.eval()  
    sentiments = []
    scores = []

    with torch.no_grad():
        for chunk in chunks:
            inputs = torch.tensor(chunk).unsqueeze(0)  
            

            inputs = torch.nn.functional.pad(inputs, (0, 512 - inputs.shape[1]), value=tokenizer.pad_token_id)
            
            outputs = model(inputs)
            logits = outputs.logits
            sentiment = torch.argmax(logits, dim=1).numpy()[0]  
            score = torch.softmax(logits, dim=1).max().item()  

            sentiments.append(sentiment)
            scores.append(score)

    return sentiments, scores


In [197]:
def aggregate_results(sentiments, scores):

    average_score_per_sentiment = {}
    for sentiment, score in zip(sentiments, scores):
        if sentiment not in average_score_per_sentiment:
            average_score_per_sentiment[sentiment] = []
        average_score_per_sentiment[sentiment].append(score)
    
    for sentiment in average_score_per_sentiment:
        average_score_per_sentiment[sentiment] = sum(average_score_per_sentiment[sentiment]) / len(average_score_per_sentiment[sentiment])
    
    dominant_sentiment = max(average_score_per_sentiment, key=average_score_per_sentiment.get)
    return dominant_sentiment, average_score_per_sentiment[dominant_sentiment]


In [253]:
df['articleInfo'][1]

'(0:30) - Learning from Warren Buffett’s Mistakes (11:10) - Stocks To Keep On Your Watchlist (26:00) - Episode Roundup: IBM, AMZN, CVX, BAC, AAPL Podcast@Zacks.com  Welcome to Episode #363 of the Value Investor Podcast. Everyone makes mistakes in investing, even the world’s greatest living investor, Warren Buffett. He’s been investing for over 80 years so there are bound to be a few blow ups along the way. What can we learn from his biggest mistakes? 1.\xa0\xa0\xa0\xa0\xa0\xa0 Buying in a difficult industry where there are few moats 2.\xa0\xa0\xa0\xa0\xa0\xa0 Holding the company too long, even when the business has changed or its not performing 3.\xa0\xa0\xa0\xa0\xa0\xa0 Buying an expensive stock Berkshire Hathaway first bought shares of IBM in 2011 around $170 a share. By 2018, Berkshire had sold out of the investment in the range of $140 to $150. Berkshire took a loss. Buffett later said IBM wasn’t performing as he thought it would and that’s why he got out. But it took him 8 years t

In [259]:
def full_window(text, tokenizer, model):
    chunks = sliding_window(text, tokenizer)
    sentiments, scores = process_chunks(chunks, tokenizer, model)
    return aggregate_results(sentiments, scores)

In [261]:
df['sentiment'] = df['articleInfo'].apply(lambda news: full_window(news, tokenizer, model)[0])
df['sentiment probability']  = df['articleInfo'].apply(lambda news: full_window(news, tokenizer, model)[1])

In [255]:
df['sentiment'].value_counts()

negative    125
positive     76
neutral      66
Name: sentiment, dtype: int64

In [264]:
full_window(df['articleInfo'][0], tokenizer, model)

(2, 0.7786687016487122)

In [262]:
df['sentiment'].value_counts()

2    196
1     40
0     31
Name: sentiment, dtype: int64